In [1]:
import numpy
import cv2
import os
from tqdm import tqdm

In [2]:
path = "../input/dynamic-sign-language-gestures-01/all"
image_size = 256
frame_rate = 0.1

def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (image_size, image_size))
    return [hasFrames, image]

In [4]:
videos_list = sorted(os.listdir(path))[:500]
new_list = []

for xyz in tqdm(range(10)):
    mno = xyz*50
    for index in range(mno, mno+50):
        new_list.append(videos_list[index])

100%|██████████| 10/10 [00:00<00:00, 45739.41it/s]


In [14]:
frames = []
labels = []

for video in tqdm(new_list):
#     vidcap = cv2.VideoCapture(path + '/' + video)
#     sec = 0
#     mini = []
#     success, frame = getFrame(sec)
#     while success:
#         mini.append(frame)
#         sec = sec + frame_rate
#         sec = round(sec, 2)
#         success, frame = getFrame(sec)
        
#     frames.append(mini[-8:])
    labels.extend([int(video[:3]) -1]*8)

100%|██████████| 500/500 [00:00<00:00, 385718.59it/s]


In [15]:
from tensorflow.keras.utils import to_categorical
# frames = numpy.array(frames)
frames = numpy.load('../input/aslframes/last8-10classes.npy')
labels = numpy.array(labels)
cat_labels = to_categorical(labels)

In [20]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(frames, labels, test_size=0.1,
                                                   shuffle=True, random_state=42, stratify=labels)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=30,
    width_shift_range=0.2,
#     height_shift_range=0.2,
    zoom_range=[0.75,1.25])
train_datagen.fit(x_train)

test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=30,
    width_shift_range=0.2,
#     height_shift_range=0.2,
    zoom_range=[0.75,1.25])
test_datagen.fit(x_test)

In [ ]:
import tensorflow
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import GlobalMaxPooling2D, Dense, Dropout
from tensorflow.keras.models import Model

model = Xception(include_top=False)
embed = GlobalMaxPooling2D()(model.output)
layer = Dropout(0.4)(embed)
layer = Dense(1024, activation='relu')(layer)
layer = Dropout(0.4)(layer)
layer = Dense(256, activation='relu')(layer)
layer = Dense(64, activation='relu')(layer)
output = Dense(10, activation='softmax')(layer)

model = Model(inputs=model.inputs, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpoint = tensorflow.keras.callbacks.ModelCheckpoint(filepath='model.h5', monitor='val_accuracy',
                                                       save_best_only=True, verbose=1)
earlyStop = tensorflow.keras.callbacks.EarlyStopping(patience=16, monitor='val_accuracy',
                                                    restore_best_weights=True, verbose=1)
reduceLR = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, 
                                                patience=4, verbose=1)

# model.fit(train_datagen.flow(x_train, y_train, batch_size=32),
#           steps_per_epoch=len(x_train) / 32, epochs=128, verbose=1,
#           validation_data=(test_datagen.flow(x_test, y_test, batch_size=32)),
#           callbacks=[checkpoint, earlyStop, reduceLR])

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=128,
          steps_per_epoch=len(x_train) / 32, verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[checkpoint, earlyStop, reduceLR])

In [ ]:
y_pred = model.predict(x_test)
from sklearn.metrics import confusion_matrix, accuracy_score
acc = accuracy_score(y_test.argmax(axis=1), y_pred.argmax(axis=1))
cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
cm

In [ ]:
vidcap = cv2.VideoCapture('../input/dynamic-sign-language-gestures-01/all/001_001_004.mp4')
sec = 0
mini = []
success, frame = getFrame(sec)
while success:
    mini.append(frame)
    sec = sec + frame_rate
    sec = round(sec, 2)
    success, frame = getFrame(sec)

In [ ]:
mini = numpy.array(mini[-8:])
y_pred = model.predict(mini)
y_pred = y_pred.argmax(axis=1)
y_pred

In [21]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2]*x_train.shape[3])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2]*x_test.shape[3])

In [22]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model = XGBClassifier()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
acc = accuracy_score(y_test, y_pred)
acc

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:23:33] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


1.0

In [23]:
from sklearn.metrics import cohen_kappa_score
kappa = cohen_kappa_score(y_test, y_pred, weights='quadratic')

In [25]:
path = "../input/team-data/Feb 2021/"
image_size = 128
frame_rate = 0.1

def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (image_size, image_size))
    return [hasFrames, image]

In [27]:
files = sorted(os.listdir(path))
new_list = []

for xyz in tqdm(files):
    for mno in os.listdir(path+xyz+'/'):
        new_list.append(path+xyz+'/'+mno)

100%|██████████| 10/10 [00:00<00:00, 288.86it/s]


In [30]:
for video in new_list:
    cap = cv2.VideoCapture(video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count/fps
    print(video, '-' ,duration)

../input/team-data/Feb 2021/00/thankyou-4.avi - 2.7333333333333334
../input/team-data/Feb 2021/00/thankyou-17.mp4 - 2.8
../input/team-data/Feb 2021/00/thankyou-15.mp4 - 2.066666666666667
../input/team-data/Feb 2021/00/thankyou-10.mp4 - 2.533333333333333
../input/team-data/Feb 2021/00/thankyou-6.mp4 - 2.4
../input/team-data/Feb 2021/00/thankyou-9.mp4 - 2.533333333333333
../input/team-data/Feb 2021/00/thank-you-1.mp4 - 2.3023000000000002
../input/team-data/Feb 2021/00/thank you-3.mp4 - 3.003
../input/team-data/Feb 2021/00/thankyou-7.mp4 - 4.866666666666666
../input/team-data/Feb 2021/00/thankyou-13.mp4 - 2.066666666666667
../input/team-data/Feb 2021/00/thankyou-19.mp4 - 2.2
../input/team-data/Feb 2021/00/thankyou-18.mp4 - 2.1333333333333333
../input/team-data/Feb 2021/00/thankyou-14.mp4 - 2.3333333333333335
../input/team-data/Feb 2021/00/thank you-2.mp4 - 1.7934583333333332
../input/team-data/Feb 2021/00/thankyou-20.mp4 - 1.9333333333333333
../input/team-data/Feb 2021/00/thankyou-22.mp4 

In [ ]:
frames = []
labels = []

for video in tqdm(new_list):
    vidcap = cv2.VideoCapture(video)
    sec = 0
    mini = []
    success, frame = getFrame(sec)
    while success:
        mini.append(frame)
        sec = sec + frame_rate
        sec = round(sec, 2)
        success, frame = getFrame(sec)
        
    mini = mini[-12:]
    frames.extend(mini)
    labels.extend([int(video[28:30])]*len(mini))

 16%|█▌        | 18/116 [00:09<00:42,  2.30it/s]

In [ ]:
frames = numpy.array(frames)
labels = numpy.array(labels)

In [48]:
print(frames.shape)
print(labels.shape)

(1391, 128, 128, 3)

In [39]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(frames, labels, test_size=0.1,
                                                   shuffle=True, random_state=42, stratify=labels)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


ValueError: Found input variables with inconsistent numbers of samples: [116, 1392]

In [ ]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2]*x_train.shape[3])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2]*x_test.shape[3])

In [ ]:
model = XGBClassifier()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
acc = accuracy_score(y_test, y_pred)
acc